In [ ]:
import numpy as np
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
%matplotlib inline

import torch, torchvision
from torchvision import datasets, transforms

from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split

In [ ]:
torch.manual_seed(69)

In [ ]:
train_data = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())
train_data, valid_data = train_test_split(train_data, test_size=0.1)
test_data = datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor())

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=8, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=8, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=8, shuffle=True)

In [ ]:
train_dataiter = iter(train_loader)
test_dataiter = iter(test_loader)


In [ ]:
imgs, labels = next(train_dataiter)

In [ ]:
label_to_name = {
    0: "Самолет",
    1: "Авто",
    2: "Птица",
    3: "Кот",
    4: "Олень",
    5: "Собака",
    6: "Лягушка",
    7: "Лошадь",
    8: "Корабль",
    9: "Грузовик"
}

In [ ]:
def show(imgs, labels):
  f, axes = plt.subplots(1, 8, figsize=(30, 30))
  for i, axis in enumerate(axes):
    axes[i].imshow(np.squeeze(np.transpose(imgs[i].numpy(), (1, 2, 0))), cmap='gray')
    axes[i].set_title(label_to_name[int(labels[i].numpy())])
  plt.show()


In [ ]:
show(imgs, labels)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
class ToVect(nn.Module):
  def forward(self, img):
    return img.view(img.size(0), -1)

Рассмотрим несколько вариантов упрощенной архитектуры перед тем, как усложнить и доучить ее

In [ ]:
ks = 3
cc = 2

class CNN(nn.Module): #один пулинг
    def __init__(self, num_classes=10):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 20, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(20),
            nn.ReLU())
        self.layer2 = nn.Sequential(
            nn.Conv2d(20, 20, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(20),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(20, 40, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(40),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(40, 40, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(40),
            nn.ReLU(),
            # nn.MaxPool2d(kernel_size = 2, stride = 2)
            )
        self.layer5 = nn.Sequential(
            nn.Conv2d(40, 40, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(40),
            nn.ReLU())
        self.layer6 = nn.Sequential(
            nn.Conv2d(40, 40, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(40),
            nn.ReLU())
        self.layer7 = nn.Sequential(
            nn.Sequential(
            nn.Conv2d(40, 40, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(40),
            nn.ReLU()),

            nn.Sequential(
            nn.Conv2d(40, 40, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(40),
            nn.ReLU()),

            # nn.Sequential(
            # nn.Conv2d(40, 40, kernel_size=3, stride=1, padding=1),
            # nn.BatchNorm2d(40),
            # nn.ReLU())
        )
        self.tovect = ToVect()
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(10240, 1000),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(1000, 100),
            nn.ReLU())
        self.fc2= nn.Sequential(
            nn.Linear(100, num_classes))

    def forward(self, x):
        out = self.layer1(x).to(device)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.tovect(out)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [ ]:
ks = 3
cc = 2

class CNN2(nn.Module): #два пулинга
    def __init__(self, num_classes=10):
        super(CNN2, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 20, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(20),
            nn.ReLU())
        self.layer2 = nn.Sequential(
            nn.Conv2d(20, 20, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(20),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(20, 40, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(40),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(40, 40, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(40),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2)
            )
        self.layer5 = nn.Sequential(
            nn.Conv2d(40, 40, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(40),
            nn.ReLU())
        self.layer6 = nn.Sequential(
            nn.Conv2d(40, 40, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(40),
            nn.ReLU())
        self.layer7 = nn.Sequential(
            nn.Sequential(
            nn.Conv2d(40, 40, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(40),
            nn.ReLU()),

            nn.Sequential(
            nn.Conv2d(40, 40, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(40),
            nn.ReLU()),

            # nn.Sequential(
            # nn.Conv2d(40, 40, kernel_size=3, stride=1, padding=1),
            # nn.BatchNorm2d(40),
            # nn.ReLU())
        )
        self.tovect = ToVect()
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(2560, 1000),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(1000, 100),
            nn.ReLU())
        self.fc2= nn.Sequential(
            nn.Linear(100, num_classes))

    def forward(self, x):
        out = self.layer1(x).to(device)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.tovect(out)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [ ]:
ks = 3
cc = 2

class CNN3(nn.Module):#три пулинга
    def __init__(self, num_classes=10):
        super(CNN3, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 20, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(20),
            nn.ReLU())
        self.layer2 = nn.Sequential(
            nn.Conv2d(20, 20, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(20),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(20, 40, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(40),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(40, 40, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(40),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2)
            )
        self.layer5 = nn.Sequential(
            nn.Conv2d(40, 80, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(80),
            nn.ReLU())
        self.layer6 = nn.Sequential(
            nn.Conv2d(80, 80, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(80),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer7 = nn.Sequential(
            nn.Sequential(
            nn.Conv2d(80, 80, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(80),
            nn.ReLU()),

            nn.Sequential(
            nn.Conv2d(80, 80, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(80),
            nn.ReLU()),

            # nn.Sequential(
            # nn.Conv2d(40, 40, kernel_size=3, stride=1, padding=1),
            # nn.BatchNorm2d(40),
            # nn.ReLU())
        )
        self.tovect = ToVect()
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(1280, 1000),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(1000, 100),
            nn.ReLU())
        self.fc2= nn.Sequential(
            nn.Linear(100, num_classes))

    def forward(self, x):
        out = self.layer1(x).to(device)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.tovect(out)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [ ]:
ks = 3
cc = 2

class CNN4(nn.Module):#четыре пулинга
    def __init__(self, num_classes=10):
        super(CNN4, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 20, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(20),
            nn.ReLU())
        self.layer2 = nn.Sequential(
            nn.Conv2d(20, 20, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(20),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(20, 40, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(40),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(40, 40, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(40),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2)
            )
        self.layer5 = nn.Sequential(
            nn.Conv2d(40, 80, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(80),
            nn.ReLU())
        self.layer6 = nn.Sequential(
            nn.Conv2d(80, 80, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(80),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer7 = nn.Sequential(
            nn.Sequential(
            nn.Conv2d(80, 160, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(160),
            nn.ReLU()),

            nn.Sequential(
            nn.Conv2d(160, 160, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(160),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2)),

            nn.Sequential(
            nn.Conv2d(160, 160, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(160),
            nn.ReLU()),

            nn.Sequential(
            nn.Conv2d(160, 160, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(160),
            nn.ReLU())
        )
        self.tovect = ToVect()
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(640, 500),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(500, 100),
            nn.ReLU())
        self.fc2= nn.Sequential(
            nn.Linear(100, num_classes))

    def forward(self, x):
        out = self.layer1(x).to(device)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.tovect(out)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [ ]:
ks = 3
cc = 2

class CNN5(nn.Module):#пять пулингов
    def __init__(self, num_classes=10):
        super(CNN5, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 20, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(20),
            nn.ReLU())
        self.layer2 = nn.Sequential(
            nn.Conv2d(20, 20, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(20),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(20, 40, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(40),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(40, 40, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(40),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2)
            )
        self.layer5 = nn.Sequential(
            nn.Conv2d(40, 80, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(80),
            nn.ReLU())
        self.layer6 = nn.Sequential(
            nn.Conv2d(80, 80, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(80),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer7 = nn.Sequential(
            nn.Sequential(
            nn.Conv2d(80, 160, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(160),
            nn.ReLU()),

            nn.Sequential(
            nn.Conv2d(160, 160, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(160),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2)),

            nn.Sequential(
            nn.Conv2d(160, 320, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(320),
            nn.ReLU()),

            nn.Sequential(
            nn.Conv2d(320, 320, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(320),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2)),

            nn.Sequential(
            nn.Conv2d(320, 320, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(320),
            nn.ReLU()),

            nn.Sequential(
            nn.Conv2d(320, 320, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(320),
            nn.ReLU()),

            nn.Sequential(
            nn.Conv2d(320, 320, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(320),
            nn.ReLU()),
            # nn.Sequential(
            # nn.Conv2d(320, 320, kernel_size=3, stride=1, padding=1),
            # nn.BatchNorm2d(320),
            # nn.ReLU())
        )
        self.tovect = ToVect()
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(320, 300),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(300, 300),
            nn.ReLU())
        self.fc2= nn.Sequential(
            nn.Linear(300, num_classes))

    def forward(self, x):
        out = self.layer1(x).to(device)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.tovect(out)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [ ]:
def train(model, n_e = 15, l_r = 0.005):
  num_classes = 10
  num_epochs = n_e
  batch_size = 8
  learning_rate = l_r

  # Loss and optimizer
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)


  # Train the model
  total_step = len(train_loader)

  total_step = len(train_loader)

  for epoch in range(num_epochs):
      for i, (images, labels) in enumerate(train_loader):
          # Move tensors to the configured device
          images = images.to(device)
          labels = labels.to(device)

          # Forward pass
          outputs = model(images)
          loss = criterion(outputs, labels)

          # Backward and optimize
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

      print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                    .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

      # Validation
      with torch.no_grad():
          correct = 0
          total = 0
          for images, labels in valid_loader:
              images = images.to(device)
              labels = labels.to(device)
              outputs = model(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()
              del images, labels, outputs
          print(total, correct)
          print('Accuracy of the network on the {} validation images: {} %'.format(total, 100 * correct / total))

In [ ]:
def check(model):
  with torch.no_grad():
      correct = 0
      total = 0
      for images, labels in test_loader:
          images = images.to(device)
          labels = labels.to(device)
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
          del images, labels, outputs

      print('Accuracy of the network on the {} test images: {} %'.format(total, 100 * correct / total))

In [ ]:
model1 = CNN().to(device)
model2 = CNN2().to(device)
model3 = CNN3().to(device)
model4 = CNN4().to(device)
model5 = CNN5().to(device)

In [ ]:
train(model1)

Epoch [1/20], Step [5625/5625], Loss: 0.9653
5000 2238
Accuracy of the network on the 5000 validation images: 44.76 %
Epoch [2/20], Step [5625/5625], Loss: 1.4146
5000 2607
Accuracy of the network on the 5000 validation images: 52.14 %
Epoch [3/20], Step [5625/5625], Loss: 1.4810
5000 2881
Accuracy of the network on the 5000 validation images: 57.62 %
Epoch [4/20], Step [5625/5625], Loss: 1.0128
5000 2986
Accuracy of the network on the 5000 validation images: 59.72 %
Epoch [5/20], Step [5625/5625], Loss: 0.6591
5000 2847
Accuracy of the network on the 5000 validation images: 56.94 %
Epoch [6/20], Step [5625/5625], Loss: 0.6213
5000 2723
Accuracy of the network on the 5000 validation images: 54.46 %
Epoch [7/20], Step [5625/5625], Loss: 1.1089
5000 2963
Accuracy of the network on the 5000 validation images: 59.26 %
Epoch [8/20], Step [5625/5625], Loss: 1.5392
5000 3046
Accuracy of the network on the 5000 validation images: 60.92 %
Epoch [9/20], Step [5625/5625], Loss: 0.8504
5000 2919
A

In [ ]:
check(model1)

In [ ]:
train(model2)

In [ ]:
check(model2)

In [ ]:
train(model3)

In [ ]:
check(model3)

In [ ]:
train(model4)

In [ ]:
check(model4)

In [ ]:
train(model5)

In [ ]:
check(model5)

Так как лучше всего себя показал 3-й вариант, улучшим именно его, увеличив количество слоев изображения после свертки

In [ ]:
class TheBestOneIHope(nn.Module):
    def __init__(self, num_classes=10):
        super(TheBestOneIHope, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32 * 2, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32 * 2),
            nn.ReLU())
        self.layer2 = nn.Sequential(
            nn.Conv2d(32 * 2, 32 * 2, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32 * 2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(32 * 2, 64 * 2, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64 * 2),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(64 * 2, 64 * 2, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64 * 2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2)
            )
        self.layer5 = nn.Sequential(
            nn.Conv2d(64 * 2, 128 * 2, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128 * 2),
            nn.ReLU())
        self.layer6 = nn.Sequential(
            nn.Conv2d(128 * 2, 128 * 2, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128 * 2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer7 = nn.Sequential(
            nn.Sequential(
            nn.Conv2d(128 * 2, 128 * 2, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128 * 2),
            nn.ReLU()),

            nn.Sequential(
            nn.Conv2d(128 * 2, 128 * 2, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128 * 2),
            nn.ReLU()),

        )
        self.tovect = ToVect()
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4*4*128 * 2, 1000),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(1000, 100),
            nn.ReLU())
        self.fc2= nn.Sequential(
            nn.Linear(100, num_classes))

    def forward(self, x):
        # out = self.layer0(x).to(device)
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.tovect(out)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

best_model = TheBestOneIHope().to(device)

In [ ]:
train(best_model, 15, 0.01)
train(best_model, 15)
train(best_model, 15, 0.001)
train(best_model, 15, 0.0005)
train(best_model, 15, 0.00005)

In [ ]:
check(best_model)